In [32]:
import pandas as pd # для работы с таблицами
import psycopg2

from math       import log
from fpg_tree   import FPTree
from sqlalchemy import create_engine
from bs4 import BeautifulSoup # для обработки HTML

ENGINE_PG = create_engine('postgresql+psycopg2://sports_ru_admin@db-test:5433/sports')

# Предобработка входных данных

In [2]:
# Считываем данные из БД.

df = pd.read_sql('''
                    (
                        SELECT distinct body
                        FROM blog_posts 
                        INNER JOIN bs_tag_links  
                            ON blog_posts.id = bs_tag_links.obj_id
                        INNER JOIN tags 
                            ON tags.id = bs_tag_links.tag_id
                        WHERE tags.name IN ('Спартак')
                        LIMIT 500
                    )
                    UNION 
                    (
                        SELECT distinct body
                        FROM blog_posts 
                        INNER JOIN bs_tag_links  
                            ON blog_posts.id = bs_tag_links.obj_id
                        INNER JOIN tags 
                            ON tags.id = bs_tag_links.tag_id
                        WHERE tags.name IN ('Мартен Фуркад')
                        LIMIT 500
                    )
                    UNION 
                    (
                        SELECT distinct body
                        FROM blog_posts 
                        INNER JOIN bs_tag_links  
                            ON blog_posts.id = bs_tag_links.obj_id
                        INNER JOIN tags 
                            ON tags.id = bs_tag_links.tag_id
                        WHERE tags.name IN ('ЧМ-2018')
                        LIMIT 500
                    )
                    UNION 
                    (
                        SELECT distinct body
                        FROM blog_posts 
                        INNER JOIN bs_tag_links  
                            ON blog_posts.id = bs_tag_links.obj_id
                        INNER JOIN tags 
                            ON tags.id = bs_tag_links.tag_id
                        WHERE tags.name IN ('Роджер Федерер')
                        LIMIT 500
                    )
                ''', ENGINE_PG)

df.head()

,body,name
0,"<embed type=""application/x-shockwave-flash"" sr...",Спартак
1,<p><em><strong>В минувшие выходные завершился ...,Роджер Федерер
2,<h2>Мутко могут отстранить от футбола</h2>\n<p...,Спартак
3,<p>102-х летний Отто Фишер - житель Копейска Ч...,Спартак
4,"<p><em><strong>Экс-пятая ракетка мира, коммент...",Роджер Федерер


In [3]:
df.loc[3, 'body']

'<p>102-х летний Отто Фишер - житель Копейска Челябинской области, старейший поклонник московского &quot;Спартака&quot;. Долгожитель отправился в Москву на матч любимой команды с &quot;Динамо&quot;. Все расходы за проезд, проживание и питание взял на себя футбольный клуб. В столице копейчанина ждет насыщенная программа: экскурсия по новому стадиону &quot;Открытие Арена&quot;, билеты в VIP-зону, фотографии с любимыми спортсменами.</p>\r\n<p><iframe src="https://www.youtube.com/embed/wpOV5cdVj94" width="560" height="315">1</iframe></p> '

In [4]:
# Получим текст по следующим тегам 'p', 'li', 'h1', поскольку в остальных тегах содержательной информации 
# для дальнейшей обработки не содержится

df['Post'] = df.body.apply(lambda x: '.'.join(content.text.lower() for content in BS(x).find_all(['p', 'li', 'h1'])))
df.head()

/Users/aleksandrtager/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/aleksandrtager/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,body,name,Post
0,"<embed type=""application/x-shockwave-flash"" sr...",Спартак,
1,<p><em><strong>В минувшие выходные завершился ...,Роджер Федерер,в минувшие выходные завершился третий турнир у...
2,<h2>Мутко могут отстранить от футбола</h2>\n<p...,Спартак,"по информации dailymail, фифа приняла во внима..."
3,<p>102-х летний Отто Фишер - житель Копейска Ч...,Спартак,102-х летний отто фишер - житель копейска челя...
4,"<p><em><strong>Экс-пятая ракетка мира, коммент...",Роджер Федерер,"экс-пятая ракетка мира, комментатор «евроспорт..."


# Составление словаря


В качестве словаря можно использовать открытый корпус с проекта OpenCorpora (http://www.opencorpora.org/) 

Скачаем морфологический словарь, который представляет собой файл XML в кодировке utf-8. Также на сайте представлено описание XML файла. 
Обрабатывать файл будем с помощью стандартной библиотеки для обратоки XML - bs4.



In [6]:
with open('dict.opcorpora.xml') as f:
    content = f.readlines()

In [7]:
print('Индексы лемм в XML:')

for i, word in enumerate(content):
    if word == '<lemmata>\n' or word == '</lemmata>\n':
        print(i, word)


Индексы лемм в XML:
535 <lemmata>

389895 </lemmata>



In [8]:
# Взглянем на данные.

content[537]

'    <lemma id="2" rev="2"><l t="ёж"><g v="NOUN"/><g v="inan"/><g v="masc"/></l><f t="ёж"><g v="sing"/><g v="nomn"/></f><f t="ежа"><g v="sing"/><g v="gent"/></f><f t="ежу"><g v="sing"/><g v="datv"/></f><f t="ёж"><g v="sing"/><g v="accs"/></f><f t="ежом"><g v="sing"/><g v="ablt"/></f><f t="еже"><g v="sing"/><g v="loct"/></f><f t="ежи"><g v="plur"/><g v="nomn"/></f><f t="ежей"><g v="plur"/><g v="gent"/></f><f t="ежам"><g v="plur"/><g v="datv"/></f><f t="ежи"><g v="plur"/><g v="accs"/></f><f t="ежами"><g v="plur"/><g v="ablt"/></f><f t="ежах"><g v="plur"/><g v="loct"/></f></lemma>\n'

 Первое слово "Ёж". Из описания в XML видно, что это существительное, мужской род, единственное число и в именительном падеже. Аналогичная статистика представлена для других падежей. 

Составим словарь ключ:значение, где ключом будет исходное слово, а значением - нормальная форма слова

In [9]:
%%time

d_morph = {}

# данные по словам хрянятся в строках от 536 и до 389895
for cont in content[536:389895]:
    # Создадим объект BeautifulSoup
    xml_soup = BeautifulSoup(cont, 'xml')
    
    for word in xml_soup.find_all('f'):
        d_morph[word['t'].replace('ё', 'е')] =  xml_soup.find('l')['t'].replace('ё', 'е')

CPU times: user 8min 31s, sys: 1.87 s, total: 8min 33s
Wall time: 8min 38s


# Нормализация

In [10]:
def normalization(name):
    """
        Данная функция выполняет следующие действия с каждым постом:
            1) удаляет все символы, кроме текста, пробелов
            2) удаляет лишние пробелы
            3) нормализует каждое слово
    """
    
    letters = [i for i in 'йцукенгшщзхъфывапролджэёячсмитьбю .,']
    name = name.replace('.', '. ').replace(',', ', ').replace(',', ', ').replace('ё', 'е')
    
    name = ''.join(i for i in name if i in letters)
    name = name.replace('.', '').replace(',', '')
    
    name = ' '.join(list(set([d_morph[i] if i in d_morph.keys() else i for i in name.split(' ')])))
    name += ' '
    return name

# Отбор признаков 

In [11]:
%%time

# Нормализуем все слова в постах
df['Post_norm'] = df.Post.apply(normalization)

CPU times: user 4.36 s, sys: 331 ms, total: 4.69 s
Wall time: 4.74 s


Составим список возможных уникальных слов в тексте

In [12]:
whole_text = df.Post_norm.sum().split(' ')
                                
features = list(set(whole_text))
len(features)

53659

Как видно на 2000 текстов приходится больше 50000 разлчиных слов. Для сокращения их числа предлагаются следующие действия:

    1) Удаление "стоп" слов
    2) Удаление слов, встречающийхся реже, чем минимальная поддержка MinSupp
    3) Удаление слов, короче 2 символов

In [13]:
stop_words = ['а', 'без', 'более', 'больше', 'большой', 'будет', 'будто', 'бы', 'был', 'была',
              'были', 'было', 'быть', 'в', 'вам', 'вас', 'вдруг', 'ведь', 'весь', 'во',
              'вот', 'впрочем', 'все', 'всегда', 'всего', 'всей', 'всех', 'всю', 'вы', 'где',
              'говорить', 'год', 'да', 'даже', 'два', 'для', 'до', 'другой', 'его', 'ее',
              'ей', 'ему', 'если', 'есть', 'еще', 'ещё', 'ж', 'же', 'за', 'зачем',
              'здесь', 'знать', 'и', 'из', 'или', 'им', 'иногда', 'их', 'к', 'как',
              'какая', 'какой', 'когда', 'конечно', 'который', 'кто', 'куда', 'ли', 'лучше', 'между',
              'меня', 'мне', 'много', 'может', 'можно', 'мой', 'мочь', 'моя', 'мы', 'на',
              'над', 'надо', 'наконец', 'нас', 'наш', 'не', 'него', 'нее', 'ней', 'нельзя',
              'нет', 'ни', 'нибудь', 'никогда', 'ним', 'них', 'ничего', 'но', 'ну', 'о',
              'об', 'один', 'он', 'она', 'они', 'оно', 'оный', 'опять', 'от', 'ото',
              'перед', 'по', 'под', 'после', 'потом', 'потому', 'почти', 'при', 'про', 'раз',
              'разве', 'с', 'сам', 'свой', 'свою', 'себе', 'себя', 'сейчас', 'сказать', 'со',
              'совсем', 'та', 'так', 'такой', 'там', 'тебя', 'тем', 'теперь', 'то', 'тогда',
              'того', 'тоже', 'только', 'том', 'тот', 'три', 'тут', 'ты', 'у', 'уж',
              'уже', 'хорошо', 'хоть', 'чего', 'чем', 'через', 'что', 'чтоб', 'чтобы', 'чуть',
              'эти', 'это', 'этого', 'этой', 'этом', 'этот', 'эту', 'я']

stop_words = [d_morph[i] for i in stop_words if i in d_morph.keys()]

In [34]:
d = {}
MinSupp = 20

for feat in features:
    num_in_text = whole_text.count(feat)
    
    # Если:
    # слово втречается чаще MinSupp и длина слова > 2 и слово не "стоп" слово
    if num_in_text > MinSupp and len(feat) > 2 and feat not in stop_words:
        # То добавим в словарь слов
        d[feat] = num_in_text 

len(d.keys())

3645

Число слов сократилось почти в 10 раз, можно попробовать составить пространство признаков из них. 

Взглянем на самые популярные:

In [35]:
sorted([(val, i) for i, val in d.items()], reverse=True)[:10]

[(919, 'первый'),
 (803, 'самый'),
 (793, 'матч'),
 (781, 'время'),
 (750, 'очень'),
 (704, 'стал'),
 (694, 'игра'),
 (684, 'команда'),
 (670, 'место'),
 (666, 'последний')]

In [36]:
# Добавим столбцы в таблицу, характеризующие содержание конкретного слова в конкретном тексте
for key in list(d.keys()):
    df[key] = df.Post_norm.apply(lambda x: 1 if key in x.split(' ') else 0)
    
df.head()

,body,name,Post,Post_norm,принести,передавал,помоему,минус,нюанс,становлюсь,...,грядущий,девятый,сюрприз,удобный,вечеринка,забит,четкий,дан,море,успешный
0,"<embed type=""application/x-shockwave-flash"" sr...",Спартак,,,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,<p><em><strong>В минувшие выходные завершился ...,Роджер Федерер,в минувшие выходные завершился третий турнир у...,куда коэффициент можно сумма сенсация триумфа...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,<h2>Мутко могут отстранить от футбола</h2>\n<p...,Спартак,"по информации dailymail, фифа приняла во внима...",чиновник оценка система нарушение предварител...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,<p>102-х летний Отто Фишер - житель Копейска Ч...,Спартак,102-х летний отто фишер - житель копейска челя...,отто динамо все старейший проезд команда насы...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"<p><em><strong>Экс-пятая ракетка мира, коммент...",Роджер Федерер,"экс-пятая ракетка мира, комментатор «евроспорт...",финал сетка киргиос евроспорта еще ракетка ти...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Отберем признаки с помощью метода взаимной информации. Этот показатель измеряет количество информации о принадлежности к классу c, которую несет наличие или отсутствие термина.

In [38]:
%%time

N = df.shape[0]

for col in list(d.keys()):
    arr_MI = []
    
    for cl in df.name.drop_duplicates():
        
        N00 = df[(df[col] != 1) & (df.name != cl)].shape[0]
        N01 = df[(df[col] != 1) & (df.name == cl)].shape[0]
        N10 = df[(df[col] == 1) & (df.name != cl)].shape[0]
        N11 = df[(df[col] == 1) & (df.name == cl)].shape[0]

        Nd0 = df[df.name != cl].shape[0]
        Nd1 = df[df.name == cl].shape[0]
        N0d = df[df[col] != 1].shape[0]
        N1d = df[df[col] == 1].shape[0]
        
        try:
            
            MI = N11 / N * log(N * N11 / Nd1 / N1d, 2) + \
                    N01 / N * log(N * N01 / Nd0 / Nd1, 2) + \
                        N10 / N * log(N * N10 / Nd0 / N1d, 2) + \
                            N00 / N * log(N * N00 / Nd0 / N10, 2)

            
            arr_MI.append(MI)
        except:
            pass
    # print(arr_MI)
    if not arr_MI or (max(arr_MI) - min(arr_MI)) / max(arr_MI) < 0.3 or max(arr_MI) < 2:
        d.pop(col, None)

CPU times: user 16min, sys: 7min 31s, total: 23min 31s
Wall time: 24min 29s


In [40]:
df = df[['body', 'name', 'Post', 'Post_norm'] + list(d.keys())]

In [39]:
# Взглянем на топ уже информативных признаков

sorted([(val, i) for i, val in d.items()], reverse=True)[:10]

[(457, 'турнир'),
 (375, 'роджер'),
 (360, 'спартак'),
 (353, 'клуб'),
 (326, 'мяч'),
 (319, 'федерер'),
 (315, 'состав'),
 (309, 'многое'),
 (294, 'гонка'),
 (285, 'финал')]

# FPTree

Основная цель - избежать многократного прохода базы транзакций, поскольку чтение из БД операция затратная, а данных может быть много 

FP дерево хранит всю информацию о всех часто встречающихся наборах
Дерево простраивается достаточно быстро идальше при чтение новых записей увеличение структуры происходить не будет. Т.о. происходит очень эффективное хранение всей быза транзакций.  

In [41]:
tree = FPTree()

# Движемся по всей базу транзакций
for row_num in range(df.shape[0]):
    
    # Получаем мн-во эл-в, в i-й транзакции
    row = list(df.loc[row_num][df.loc[row_num] == 1].index)
    
    # Сортируем элементы по частоте встречаемости в тексте
    row_sorted = [j[1] for j in sorted([(d[i], i) 
                       for i in row], reverse=True)]
    
    # И добавляем в дерево
    if row_sorted:
        tree.add(row_sorted)

In [42]:
def conditional_tree(v, T, l, path = []):
    """
        Данная процедура строить условное FP дерево
        На вход подуется корень исходного дерева,
        экземпляр условного дерева и признак l
    """
      
    for child in v.children:
        
        path.append(child[0])  
        
        if child[0] == l:
            
            path = path[:-1]
            
            T.add(path)
        else:
            k, path = conditional_tree(child[1], T, l, path)
            
    path = path[:-1]  
    return v._count, path



In [43]:
def disp(self, ind=1):
    """
        Данная функция предоставляет
        простенькую визуализацию дерева.
        На вход подается корневая вершина. 
    """
    
    print ('    '*ind, self._item, ' ', self._count)
    for child in self.children:
        disp(child[1], ind+1)

In [44]:
def find(tree, key, c=0):
     """
         Данная функция находит суммарную поддержку 
         признака key в дереве tree
     """
    
    for child in tree.children:
        if child[0] == key:
            c += child[1].count
        c = find(child[1], key, c)
    return c

In [45]:
def FP_Find(T, fi, R):
    """
        Основная рекурсивная функция - находит список ассоциативных правил
        На вход получает дерево, множество признаков фи и список правил
        Возвращает список правил
    """

    # Идем по уровням дерева СНИЗУ-ВВЕРХ
    for f in [j[1] for j in sorted([(val, key) 
                                    for key, val in d.items() ])]:
        # Если признак в текузем дереве встречатся больше, чем MinSup
        if find(T.root, f) > 50:
            # Удаляем старое (неполное) множество из списка правил
            if fi in R:
                R.remove(fi)
            
            # Создаем условное FP дерево
            cond_tree = FPTree()
            conditional_tree(T.root, cond_tree, f, [])
    
            # Добавляем новое правило
            R.append(fi)

            # И рекурсивно вызываем FP_Find
            R = FP_Find(cond_tree, fi.union({f}), R)
    
    return R       

In [46]:
%%time
R  = []
fi = set()

rules = FP_Find(tree, fi, R)

CPU times: user 1min 14s, sys: 535 ms, total: 1min 15s
Wall time: 1min 19s


In [47]:
# Взглянем на список правил

rules[:5]

[{'глобус'}, {'гонкий'}, {'милош', 'раонич'}, {'милош'}, {'раонич'}]

Видно, что надо обработать получившиеся правила. Это можно сделать сделав групировку по каждому уникальному слову, взяв объединение как агрегатную функцию. 

In [48]:
arr_rule = []

# идем по всем признакам, входящий в ассоциативные правила
for word in set.union(*rules):
    arr_word = []
    
    # Идем по правилам
    for rule in rules:
        if word in rule: # Если слово встретилось
            arr_word.append(rule)  # добавляем его
            
    #  Получаем множество слов, часто встречающихся с word
    arr_rule.append(set.union(*arr_word)) 
arr_rule[:5]

[{'глобус'},
 {'гейм',
  'джокович',
  'партия',
  'подача',
  'полуфинал',
  'ракетка',
  'рафа',
  'рейтинг',
  'сет',
  'сетка',
  'теннисист',
  'федерера',
  'швейцарец',
  'энди'},
 {'антон',
  'биатлонист',
  'зачет',
  'индивидуальный',
  'лыжа',
  'медаль',
  'подиум',
  'промах',
  'рубеж',
  'секунда',
  'спринт',
  'стрельба',
  'трасса',
  'француз',
  'фуркада',
  'эстафета'},
 {'антон',
  'биатлонист',
  'зачет',
  'индивидуальный',
  'лыжа',
  'медаль',
  'норвежец',
  'подиум',
  'промах',
  'рубеж',
  'спринт',
  'стрельба',
  'трасса',
  'француз',
  'фуркада',
  'эстафета'},
 {'виктория',
  'выпуск',
  'дель',
  'джокович',
  'мария',
  'маррей',
  'мартин',
  'новак',
  'полуфинал',
  'ракетка',
  'рафа',
  'сет',
  'теннисист',
  'теннисный',
  'томаш',
  'уильямс',
  'федерера',
  'хуан',
  'шарапова',
  'энди'}]

Получили для каждого слова множество слов, с которыми оно часто встречается. Теперь сделаем групировку уже по наболее похожим правилам. 

In [49]:
# Остортивуем список по длинне множеств в нем
arr_rule = sorted(arr_rule, key = len, reverse=True)

# ДЛя каждого правила
for i, rule in enumerate(arr_rule):
    
    # Для всех следующих правил (меньшей длины)
    for new_rule in arr_rule[i+1:]:
        # Если правило состоит из менее 3-х признаков или явл-ся подмножеством rule, то удаляем его
        if new_rule in arr_rule and (len(new_rule) < 3 or new_rule.issubset(arr_rule[i])):
            arr_rule.remove(new_rule)
            
        # Здесь использована достаточно эмпирическая оценка схожести правил, если мощность пересечения двух правил 
        # меньше чем в 2 раза меньше длины нового правила, то берем их объединение
        elif (len(new_rule) - len(new_rule.intersection(rule))) / len(new_rule) < 0.5:
            arr_rule[i] = arr_rule[i].union(new_rule)
            arr_rule.remove(new_rule)
            
len(arr_rule)

4

В итоге получаем 4 правила, два из них (2 и 3) вероятно относятся к тэгу "Федерер", но такое разделение имеет право на существование, поскольку тексты действительно сильно отличаются по тематике, одни связаны непосредственно со спортсменом, другие с жизнью всех тенисистов. 

Также стоит отметить, что нет правила для класса ЧМ2018, возможно это связанно с отсутсвтие сильно информативных слов (выделяющих его от други классов) в тестах с этим тегом.

In [50]:
for i in arr_rule:
    print(i, '\n\n')

{'французский', 'зачет', 'секунда', 'медаль', 'эстафета', 'фуркада', 'промах', 'симона', 'лыжа', 'лыжный', 'трасса', 'стрельба', 'норвежец', 'финиш', 'преследование', 'эмиль', 'индивидуальный', 'подиум', 'биатлон', 'биатлонист', 'спринт', 'золото', 'стреляю', 'француз', 'антон', 'рубеж'} 


{'шлем', 'сетка', 'новак', 'ракетка', 'гейм', 'дель', 'федерера', 'полуфинал', 'маррей', 'теннисист', 'шарапова', 'серен', 'джокович', 'ролан', 'рейтинг', 'теннис', 'теннисный', 'выпуск', 'уильямс', 'федерером', 'швейцарец', 'хуан', 'мартин', 'сет', 'подача', 'виктория', 'томаш', 'энди', 'гаррос', 'партия', 'рафа', 'рафаэль', 'мария'} 


{'елена', 'шарапова', 'ана', 'анастасия', 'мартин', 'джокович', 'новак', 'дель', 'иванович', 'виктория', 'выпуск', 'уильямс', 'мария', 'хуан'} 


{'динамо', 'зенит', 'вратарь', 'забил', 'ворота', 'локомотив', 'гол', 'лига', 'оборона', 'защитник', 'цска'} 


